In [9]:
def replace_char_at_index(string, index, new_char):
  return string[:index] + new_char + string[index+1:]

In [10]:
def EliminateImplication(expression):
    elements = expression.split(" ")
    for i in range(len(elements)):
        if elements[i] == "⇒":
            elements[i] = "∨"
            if elements[i - 1] == "]":
                elements.insert(elements.index("["), "¬")
            else:
                elements.insert(i - 1, "¬")
    result = " ".join(elements)
    return result


formula1 = "P ⇒ Q"
eliminated_implication = EliminateImplication(formula1)
print("After eleminate imolications : ") 
print(eliminated_implication)

After eleminate imolications : 
¬ P ∨ Q


In [11]:
# 2. Move negation inward (Demorgan Law)
def MoveNegationInward(formula):
    new_formula = ""
    i = 0
    while i < len(formula):
        if formula[i] == '¬':
            if formula[i + 1] == '(':
                # Move negation inside the parentheses
                j = i + 2
                count_open = 1
                while count_open > 0:
                    if formula[j] == '(':
                        count_open += 1
                    elif formula[j] == ')':
                        count_open -= 1
                    j += 1
                inside_parentheses = formula[i + 2:j - 1]
                new_formula += "(" + MoveNegationInward(inside_parentheses) + ")"
                i = j
            else:
                # Apply De Morgan's law
                if formula[i + 1] == '¬':
                    new_formula += formula[i + 2]
                elif formula[i + 1] == '(':
                    j = i + 2
                    count_open = 1
                    while count_open > 0:
                        if formula[j] == '(':
                            count_open += 1
                        elif formula[j] == ')':
                            count_open -= 1
                        j += 1
                    inside_parentheses = formula[i + 2:j - 1]
                    new_formula += "¬(" + MoveNegationInward(inside_parentheses) + ")"
                    i = j
                else:
                    new_formula += "¬" + formula[i + 1]
                i += 2
        else:
            new_formula += formula[i]
            i += 1
    return new_formula

In [12]:
# 3. Remove double-not.
def RemoveDoubleNot(formula):
    new_formula = ""
    i = 0
    while i < len(formula):
        if formula[i:i+2] == '¬¬':
            # Remove double negation
            i += 2
        else:
            new_formula += formula[i]
            i += 1
    return new_formula

In [13]:
# 4. Standardize variable scope.
def StandardizeVariableScope(formula):
    global counter
    counter = 97
    for i in range(len(formula)):
        if formula[i] == '(' and formula[i - 1].isupper():
            for j in range(i + 2, len(formula)):
                if formula[i + 1] == formula[j]:
                    counter += 1
                    if counter == 99 or counter == 102:
                        counter += 1
                    if (formula[i - 3] in ("∃", "∀") and formula[i - 2] == formula[i + 1]):
                        formula = replace_char_at_index(formula, i - 2, chr(counter))
                    if (formula[i - 4] in ("∃", "∀") and formula[i - 3] == formula[i + 1]):
                        formula = replace_char_at_index(formula, i - 3, chr(counter))
                    formula = replace_char_at_index(formula, i + 1, chr(counter))
                    break
        if formula[i] == ',':
            for j in range(len(formula)):
                if (formula[i + 1] == formula[j]):
                    counter += 1
                    if (formula[i - 5] in ("∃", "∀") and formula[i - 4] == formula[i + 1]):
                        formula = replace_char_at_index(formula, i - 4, chr(counter))
                    formula = replace_char_at_index(formula, i + 1, chr(counter))
                    break
    return formula

In [14]:
# 5. The prenex form (obtained by moving all quantifiers to the left of the formula.)
def MoveQuantifiersToLeft(formula):
    # Extract quantifiers and matrix
    quantifiers = []
    matrix = ""
    i = 0
    while i < len(formula):
        if formula[i] in ("∀", "∃"):
            quantifier = ""
            while i < len(formula) and formula[i] in ("∀", "∃"):
                quantifier += formula[i]
                i += 1
            quantifiers.append(quantifier)
        else:
            matrix = formula[i:]
            break
        i += 1

    # Construct prenex form
    prenex_form = "".join(quantifiers) + matrix

    return prenex_form

In [15]:
# 6. Skolemization for existential quantifiers.
def Skolemization(formula):
    global counter
    counter = 97
    count = 0
    flag = False
    for i in range(len(formula)):
        if(formula[i] == '∃'):
            count += 1
            for j in range(len(formula)):
                if (formula[j] == '∀'):
                    flag = True
                    for k in range(i+3,len(formula)):
                        if(formula[i+1] == formula[k]):
                            formula = replace_char_at_index(formula, k, "f("+formula[j+1]+")")

            if flag == False:
                for k in range(i + 3, len(formula)):
                    if (formula[i + 1] == formula[k]):
                        formula = replace_char_at_index(formula, k, 'c')
    for i in range(count):
        formula = replace_char_at_index(formula, formula.index("∃")+1, '')
        formula = replace_char_at_index(formula, formula.index("∃"), '')
    return formula

In [16]:
# 7. Eliminate universal quantifiers.
def EliminateUniversalQuantifiers(formula):
    global counter
    for i in range(len(formula)):
        if formula[i] == '(' and formula[i - 1].isupper():
            for j in range(i + 2, len(formula)):
                if formula[i + 1] == formula[j]:
                    counter += 1
                    if counter == 99 or counter == 102:
                        counter += 1
                    if (formula[i - 3] == "∀" and formula[i - 2] == formula[i + 1]):
                        formula = replace_char_at_index(formula, i - 2, chr(counter))
                    if (formula[i - 4] == "∀" and formula[i - 3] == formula[i + 1]):
                        formula = replace_char_at_index(formula, i - 3, chr(counter))
                    formula = replace_char_at_index(formula, i + 1, chr(counter))
                    break
        if formula[i] == ',':
            for j in range(len(formula)):
                if (formula[i + 1] == formula[j]):
                    counter += 1
                    if (formula[i - 5] == "∀" and formula[i - 4] == formula[i + 1]):
                        formula = replace_char_at_index(formula, i - 4, chr(counter))
                    formula = replace_char_at_index(formula, i + 1, chr(counter))
                    break
    return formula

In [17]:
# 8. Convert to conjunctive normal form
def to_cnf(formula):
    formula = EliminateImplication(formula)
    formula = MoveNegationInward(formula)
    return formula

In [18]:
# 9. Turn conjunctions into clauses in a set, and rename variables so that no clause shares the same variable name
def ToClauseSet(formula):
    # Split the formula into individual clauses based on conjunctions
    clauses = formula.split('∧')

    # Rename variables within each clause to ensure uniqueness
    unique_var_counter = 97
    variable_mapping = {}  # Dictionary to store variable mappings

    def RenameVariablesInClause(clause):
        nonlocal unique_var_counter
        nonlocal variable_mapping

        renamed_clause = ""
        for char in clause:
            if char.isalpha():
                if char not in variable_mapping:
                    variable_mapping[char] = chr(unique_var_counter)
                    unique_var_counter += 1
                renamed_clause += variable_mapping[char]
            else:
                renamed_clause += char
        return renamed_clause

    # Apply renaming to each clause
    renamed_clauses = [RenameVariablesInClause(clause.strip()) for clause in clauses]

    # Convert clauses into a set
    clause_set = set(renamed_clauses)

    return clause_set

In [19]:
# 10. Rename variables in clauses so that each clause has a unique variable name
def RenameVariablesInClauses(clauses):
    unique_var_counter = 97
    variable_mapping = {}  # Dictionary to store variable mappings

    renamed_clauses = []

    for clause in clauses:
        def RenameVariablesInClauses():
            nonlocal unique_var_counter
            nonlocal variable_mapping

            renamed_clause = ""
            for char in clause:
                if char.isalpha():
                    if char not in variable_mapping:
                        variable_mapping[char] = chr(unique_var_counter)
                        unique_var_counter += 1
                    renamed_clause += variable_mapping[char]
                else:
                    renamed_clause += char
            return renamed_clause

        renamed_clause = RenameVariablesInClauses()
        renamed_clauses.append(renamed_clause)

    return renamed_clauses



In [20]:
# Test all functions together
initial_formula = "(∀x)(∃y)(P(x, y) ⇒ Q(z))"
transformed_formula = EliminateImplication(initial_formula)
transformed_formula = MoveNegationInward(transformed_formula)
transformed_formula = RemoveDoubleNot(transformed_formula)
transformed_formula = StandardizeVariableScope(transformed_formula)
transformed_formula = MoveQuantifiersToLeft(transformed_formula)
transformed_formula = Skolemization(transformed_formula)
transformed_formula = EliminateUniversalQuantifiers(transformed_formula)
transformed_formula = to_cnf(transformed_formula)
transformed_clause_set = ToClauseSet(transformed_formula)
renamed_clauses = RenameVariablesInClauses(transformed_clause_set)
print(renamed_clauses)


['(∀a)()(b(c,d¬ e(a)) ∨ f(g))']
